<a href="https://colab.research.google.com/github/lnwprameza007/bsc_dpdm24/blob/main/midterm_dpdm24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## การเตรียมข้อมูลประกอบด้วย กำจัด outlier, แก้ missing, normalization, one-hot-encoder และ อื่นๆ ที่นักศึกษาเห็นว่าเหมาะสม

# 1 เตรียมข้อมูลเพื่อทำนายราคาบ้าน (15 คะแนน)
เตรียมข้อมูลเพื่อทำนายราคาบ้านมือสอง ราคาประเมินบ้าน และ คะแนนทรัพย์ จาก [google drive](https://drive.google.com/drive/folders/1uIcUFY1yldIq-A3wijh2n2N6az-EljIe?usp=sharing) โดยข้อมูลที่เตรียมต้องประกอบด้วย columns ต่อไปนี้

* ```'renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking', 'sell_price', 'district', 	'sub_district', 'zone'``` จากไฟล์ "Data House" - บ้านมือ 2 (5 คะแนน)

* ```จำนวน 7-11 และ โรงเรียน ในรัศมี 1 กิโลเมตร (ตั้งชื่อ 'd1000-7/11' และ 'd1000-school')``` จากไฟล์ "7-eleven.json" และ "school.json" โดยคำนวณจากพิกัด latitute longitude ระหว่างบ้าน และ  7-11 หรือ โรงเรียน ใกล้เคียงของบ้านแต่ละ id ในไฟล์ json (id บ้านหลังไหนที่มีมีข้อมูลแปลว่าไม่มี 7-11 หรือ โรงเรียนอยู่ในรัศมี 5 กิโลเมตร) (7 คะแนน)

* ```ประเมินราคาซื้อขาย และ คะแนนทรัพย์ (ตั้งชื่อ 'estimator' และ 'score')``` จากไฟล์ "คำตอบนักประเมิน(ID)" - บ้าน โดยราคาประเมินกับคะแนนทรัพย์ อ้างอิงของนักประเมิน ID=01 คนเดียว (3 คะแนน)

# 2 จากตารางในข้อ 1 ตอบคำถามต่อไปนี้ (5 คะแนน)
1. zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)
2. บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่ (2 คะแนน)
3. ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)

#Read data

In [ ]:
import pandas as pd
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

spreadsheet_id = '1QO6OiGqyr8i78MVnjyLBBtr5Y8AoBM1tHZRg0Ud5CW4'
worksheet_name = 'บ้านมือ2'  # Replace with the actual sheet name

# Open the spreadsheet and select the worksheet
worksheet = gc.open_by_key(spreadsheet_id).worksheet(worksheet_name)

# Get all values from the worksheet
rows = worksheet.get_all_values()

# Convert the data to a Pandas DataFrame
home2 = pd.DataFrame.from_records(rows[1:], columns=rows[0]) # Assuming the first row contains column headers
home2

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,home improvement,...,furniture,building_number,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,...,ไม่มี,,,,,,,,,
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,...,ไม่มี,,,,,,,,,
2,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,,,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,...,ไม่ระบุ,,,,,,,,,
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,,,,,,,,,
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,20460P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06026,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06027,,,ขอนแก่น,บ้านฝาง,หนองบัว,บ้านเดี่ยว,no,...,ไม่ระบุ,,,,,,,,,
1652,20462P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06033,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06034,,,ขอนแก่น,หนองเรือ,โนนทอง,บ้านเดี่ยว,no,...,ไม่ระบุ,,,,,,,,,
1653,20463P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07581,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07582,,,ขอนแก่น,แวงน้อย,ก้านเหลือง,บ้านเดี่ยว,no,...,ไม่ระบุ,,,,,,,,,
1654,20466P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10402,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10403,,,ขอนแก่น,ชุมแพ,วังหินลาด,บ้านเดี่ยว,no,...,ไม่ระบุ,,,,,,,,,


In [ ]:
worksheet_name2 = 'บ้านมือ1'

# Open the spreadsheet and select the worksheet
worksheet = gc.open_by_key(spreadsheet_id).worksheet(worksheet_name2)

# Get all values from the worksheet
rows = worksheet.get_all_values()

# Convert the data to a Pandas DataFrame
home1 = pd.DataFrame.from_records(rows[1:], columns=rows[0]) # Assuming the first row contains column headers
home1

,id,parent_id,project_name,province,district,sub_district,zip_code,zone_id,zone,property_type,...,cable_free,cable_none,cable_ask,position_previous,position_current,sold_out_at,is_pro_in_house,created_at,updated_at,deleted_at
0,314,41,ที สเปซ กลางเมือง,ขอนแก่น,เมืองขอนแก่น,เมืองเก่า,,5,กลางเมือง-เมืองเก่า-กุดกว้าง,บ้านเดี่ยว,...,,,,0,0,,0,44012,44602,
1,315,41,ที สเปซ กลางเมือง,ขอนแก่น,เมืองขอนแก่น,เมืองเก่า,,5,กลางเมือง-เมืองเก่า-กุดกว้าง,บ้านแฝด,...,,,,0,0,,0,44012,44602,
2,316,41,ที สเปซ กลางเมือง,ขอนแก่น,เมืองขอนแก่น,เมืองเก่า,,5,กลางเมือง-เมืองเก่า-กุดกว้าง,บ้านเดี่ยว,...,,,,0,0,,0,44012,44602,
3,317,41,ที สเปซ กลางเมือง,ขอนแก่น,เมืองขอนแก่น,เมืองเก่า,,5,กลางเมือง-เมืองเก่า-กุดกว้าง,บ้านเดี่ยว,...,,,,0,0,,0,44012,44602,
4,318,41,ที สเปซ กลางเมือง,ขอนแก่น,เมืองขอนแก่น,เมืองเก่า,,5,กลางเมือง-เมืองเก่า-กุดกว้าง,บ้านเดี่ยว,...,,,,0,0,,0,44012,44602,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,62204519412ede001495e6cb,622045edc1620300140bc3de,บ้านชีวิตดี (หนองหิน-ศิลา),ขอนแก่น,เมืองขอนแก่น,ศิลา,,,,บ้านเดี่ยว,...,,,,,,,,,,
750,62201e6fa441d30014e1f350,62201f07a441d30014e1f352,เดอะ คาล์ม วิลเลจ,ขอนแก่น,เมืองขอนแก่น,พระลับ,,,,บ้านเดี่ยว,...,,,,,,,,,,
751,5fa0e476a3326800191e29ea,5fa10335d0235b0019c19548,เดอะ เล็ก วิวล์ @พล,ขอนแก่น,พล,โจดหนองแก,,,,บ้านเดี่ยว,...,,,,,,,,,,
752,5e9d6fa69f161600121b61b2,5e9d70379f161600121b61b7,เดอะ เล็ก วิวล์ @แวงน้อย,ขอนแก่น,แวงน้อย,แวงน้อย,,,,บ้านเดี่ยว,...,,,,,,,,,,


In [ ]:
spreadsheet_id = '1A3A-Yl4Tmcq2kPWf37QG6xdRZZ-4XDWULRPfORGPrNQ'
worksheet_names = ['บ้าน', 'secondhand', 'ชีต8', 'ชีต9']

dfs = {}
for name in worksheet_names:
    try:
        worksheet = gc.open_by_key(spreadsheet_id).worksheet(name)
        rows = worksheet.get_all_values()
        df = pd.DataFrame.from_records(rows[1:], columns=rows[0])
        dfs[name] = df
        print(f"Successfully loaded worksheet: {name}")
    except gspread.exceptions.WorksheetNotFound:
        print(f"Worksheet '{name}' not found in the spreadsheet.")

# Access the dataframes using their worksheet names as keys
# Example: Accessing the dataframe for worksheet 'บ้าน'
if 'บ้าน' in dfs:
    print(dfs['บ้าน'].head())

Successfully loaded worksheet: บ้าน
Successfully loaded worksheet: secondhand
Successfully loaded worksheet: ชีต8
Successfully loaded worksheet: ชีต9
  ID ผู้ประเมิน                              post id   ID property_type  \
0             1   https://nayoo.co/khonkaen/posts/25   25    บ้านเดี่ยว   
1             2   https://nayoo.co/khonkaen/posts/25   25    บ้านเดี่ยว   
2             1   https://nayoo.co/khonkaen/posts/33   33    บ้านเดี่ยว   
3             2   https://nayoo.co/khonkaen/posts/33   33    บ้านเดี่ยว   
4             1  https://nayoo.co/khonkaen/posts/495  495    บ้านเดี่ยว   

  ความมั่นใจในการเมินราคาทรัพย์ ราคาทรัพย์ ประเมินราคาซื้อ-ขาย  \
0                        มั่นใจ  2,890,000           2,500,000   
1                        มั่นใจ  2,890,000           2,700,000   
2                        มั่นใจ  2,600,000           2,200,000   
3                        มั่นใจ  2,600,000           2,400,000   
4                        มั่นใจ  6,050,000           5,000,000   

  

In [ ]:
home1es = dfs['บ้าน']
home2es = dfs['secondhand']
selling_type = dfs['ชีต8']
sell_price = dfs['ชีต9']

In [ ]:
sell_price

,id,sell_price
0,1,390000
1,4,930000
2,6,6600000
3,7,710000
4,9,13000000
...,...,...
14127,33533,891000
14128,33534,935000
14129,33536,4500000
14130,33540,950000


In [ ]:
home1es

,ID ผู้ประเมิน,post id,ID,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์
0,1,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,"2,890,000","2,500,000","5,000-15,000",,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1,2,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,"2,890,000","2,700,000","โครงการนี้ราคาเสนอขาย 2,500,000-3,000,000 บาท/...",,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
2,1,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,"2,600,000","2,200,000","5,000-15,000",,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
3,2,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,"2,600,000","2,400,000","ราคาเสนอขายประมาณ 2,200,000 - 2,500,000 บาท/หลัง",,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
4,1,https://nayoo.co/khonkaen/posts/495,495,บ้านเดี่ยว,มั่นใจ,"6,050,000","5,000,000","5,000-15,000",,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
...,...,...,...,...,...,...,...,...,...,...
1912,2,https://nayoo.co/khonkaen/posts/17867,17867,บ้านเดี่ยว,มั่นใจ,"965,000","550,000","ราคาเสนอขาย 2,000 - 3,000 บาท/ตารางวา",,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1913,2,https://nayoo.co/khonkaen/posts/17873,17873,บ้านเดี่ยว,มั่นใจ,"1,000,000","1,000,000","ราคาเสนอขาย 2,500 - 3,000 บาท/ตารางวา",,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1914,2,https://nayoo.co/khonkaen/posts/17874,17874,บ้านเดี่ยว,มั่นใจ,"10,873,000","10,000,000","ราคาเสนอขาย 12,000 - 15,000 บาท/ตารางวา",,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1915,2,https://nayoo.co/khonkaen/posts/17876,17876,บ้านเดี่ยว,มั่นใจ,"1,000,000","1,200,000",ราคาเสนอขาย 1.2-1.5 ล้านบาท/หลัง,,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import json

f = open('/content/drive/My Drive/bsc_dpdm24/data/school.json')
school_data = json.load(f)

# Create a list to store individual school records
school_records = []
for school_id, records in school_data.items():
    for record in records:
        # Add school_id to each record
        record['school_id'] = school_id
        school_records.append(record)

# Create DataFrame from the list of school records
school = pd.DataFrame(school_records)
school

,business_status,geometry,icon,icon_background_color,icon_mask_base_uri,name,photos,place_id,rating,reference,scope,types,user_ratings_total,vicinity,school_id,plus_code,opening_hours,permanently_closed
0,OPERATIONAL,"{'location': {'lat': 16.1311875, 'lng': 102.65...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Ban La Wa Health Center,"[{'height': 3456, 'html_attributions': ['<a hr...",ChIJW5GGVOuCGDERn2dgufu1KOU,4.5,ChIJW5GGVOuCGDERn2dgufu1KOU,GOOGLE,"[school, point_of_interest, establishment]",8,"4MJ2+F5, Mueang Phia, Ban Phai District",1,NaN,NaN,NaN
1,OPERATIONAL,"{'location': {'lat': 16.1350641, 'lng': 102.62...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Ban Wang Woen Kut Lom School,"[{'height': 3682, 'html_attributions': ['<a hr...",ChIJ1QvkXLaCGDERc-lFOXnj0NY,5.0,ChIJ1QvkXLaCGDERc-lFOXnj0NY,GOOGLE,"[primary_school, school, point_of_interest, es...",3,"8 Soi Si Bunrueang, Chonnabot, Chonnabot District",1,"{'compound_code': '4JPH+2J Si Bun Rueang, Chon...",NaN,NaN
2,OPERATIONAL,"{'location': {'lat': 16.1482073, 'lng': 102.62...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,ศูนย์พัฒนาเด็กเล็กบ้านหนองหัวช้าง,NaN,ChIJ3x63JVOdGDERRmIYy0FvMus,0.0,ChIJ3x63JVOdGDERRmIYy0FvMus,GOOGLE,"[school, point_of_interest, establishment]",0,"4JXC+78J, Nong Paen, Mancha Khiri District",1,NaN,NaN,NaN
3,OPERATIONAL,"{'location': {'lat': 16.1279195, 'lng': 102.61...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,บ้านหนองหวาย,"[{'height': 1588, 'html_attributions': ['<a hr...",ChIJG5AU2QCdGDERoKsuWds9l1w,0.0,ChIJG5AU2QCdGDERoKsuWds9l1w,GOOGLE,"[school, point_of_interest, establishment]",0,"4JH7+5G8, Si Bun Rueang, Chonnabot District",1,NaN,NaN,NaN
4,OPERATIONAL,"{'location': {'lat': 16.6744524, 'lng': 102.91...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Ban Nong Bua Ban School,"[{'height': 1327, 'html_attributions': ['<a hr...",ChIJ3z46bb7wIjERqVRwLwU9thY,5.0,ChIJ3z46bb7wIjERqVRwLwU9thY,GOOGLE,"[school, point_of_interest, establishment]",1,"MWF9+QHX, Sai Mun, Nam Phong District",4,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36605,OPERATIONAL,"{'location': {'lat': 13.7493223, 'lng': 100.65...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Lamsalee School Bangkapi Bangkok,"[{'height': 2296, 'html_attributions': ['<a hr...",ChIJU39x_7xhHTERRm3h23OzNSU,4.2,ChIJU39x_7xhHTERRm3h23OzNSU,GOOGLE,"[school, point_of_interest, establishment]",18,"183 Krung Thep Kritha Rd, Hua Mak, Bang Kapi D...",30362P,"{'compound_code': 'PMX2+Q8 Bangkok', 'global_c...",NaN,NaN
36606,OPERATIONAL,"{'location': {'lat': 13.7332085, 'lng': 100.64...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,piyajitvittaya school,NaN,ChIJL0tIrbVhHTERF3fLJNvGKJ0,0.0,ChIJL0tIrbVhHTERF3fLJNvGKJ0,GOOGLE,"[school, point_of_interest, establishment]",0,"4 Phatthanakan Rd, Suan Luang",30362P,"{'compound_code': 'PJMV+7X Bangkok', 'global_c...",{'open_now': True},NaN
36607,OPERATIONAL,"{'location': {'lat': 13.7645946, 'lng': 100.63...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Khaisri Pramot Anusorn School,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJOVwBA-5hHTERSLgjF1AqtG4,3.4,ChIJOVwBA-5hHTERSLgjF1AqtG4,GOOGLE,"[primary_school, school, point_of_interest, es...",19,"136 Lat Phrao 134 Alley, Bangkok",30362P,"{'compound_code': 'QJ7P+RH Bangkok', 'global_c...",NaN,NaN
36608,OPERATIONAL,"{'location': {'lat': 13.733252, 'lng': 100.640...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Khlo

In [ ]:
f = open('/content/drive/My Drive/bsc_dpdm24/data/7-eleven.json')
seven_data = json.load(f)

# Create a list to store individual 7-eleven records
seven_records = []
for seven_id, records in seven_data.items():
    for record in records:
        # Add seven_id to each record
        record['seven_id'] = seven_id
        seven_records.append(record)

# Create DataFrame from the list of 7-eleven records
seven = pd.DataFrame(seven_records)
seven

,business_status,geometry,icon,icon_background_color,icon_mask_base_uri,name,opening_hours,photos,place_id,rating,reference,scope,types,user_ratings_total,vicinity,seven_id,plus_code,permanently_closed
0,OPERATIONAL,"{'location': {'lat': 16.0603149, 'lng': 102.72...",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,7-Eleven,{'open_now': True},"[{'height': 3072, 'html_attributions': ['<a hr...",ChIJzbBjEk2BGDER4YEKqD5kwlI,4.0,ChIJzbBjEk2BGDER4YEKqD5kwlI,GOOGLE,"[convenience_store, store, food, point_of_inte...",15,"3P6G+4HH, Prapat Bamrung Alley, Nai Mueang, Ba...",6,NaN,NaN
1,OPERATIONAL,"{'location': {'lat': 16.0564741, 'lng': 102.72...",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,7-Eleven,{'open_now': True},"[{'height': 3120, 'html_attributions': ['<a hr...",ChIJWwEGw7KGGDERygXhoovc3Hg,3.3,ChIJWwEGw7KGGDERygXhoovc3Hg,GOOGLE,"[convenience_store, store, food, point_of_inte...",6,"197 Ratchanikunl Alley, Nai Mueang, Ban Phai D...",6,"{'compound_code': '3P4G+HP Nai Mueang, Ban Pha...",NaN
2,OPERATIONAL,"{'location': {'lat': 16.0595413, 'lng': 102.72...",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,7-Eleven,{'open_now': True},"[{'height': 4160, 'html_attributions': ['<a hr...",ChIJa8T1gu-BGDERq8cyG-vAu_o,4.0,ChIJa8T1gu-BGDERq8cyG-vAu_o,GOOGLE,"[convenience_store, store, food, point_of_inte...",1,"3P5H+RQ7, Nai Mueang, Ban Phai District",6,NaN,NaN
3,OPERATIONAL,"{'location': {'lat': 16.0751998, 'lng': 102.73...",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,7-Eleven สาขา PTTOR อำเภอบ้านไผ่ (14179),{'open_now': True},"[{'height': 1280, 'html_attributions': ['<a hr...",ChIJyyeKOR6BGDERawb3IXZz_5U,4.0,ChIJyyeKOR6BGDERawb3IXZz_5U,GOOGLE,"[convenience_store, store, food, point_of_inte...",3,"3PGP+36Q, Nai Mueang, Ban Phai District",6,NaN,NaN
4,OPERATIONAL,"{'location': {'lat': 16.070648, 'lng': 102.734...",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,7-Eleven สาขา ชุมชนหนองลุมพุก (06323),{'open_now': True},"[{'height': 2352, 'html_attributions': ['<a hr...",ChIJ9zwFukeBGDERFV0i5_PJNLc,4.0,ChIJ9zwFukeBGDERFV0i5_PJNLc,GOOGLE,"[convenience_store, store, food, point_of_inte...",1,"3PCM+7Q2, Nai Mueang, Ban Phai District",6,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25049,OPERATIONAL,"{'location': {'lat': 13.74039, 'lng': 100.6252...",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,7-Eleven สาขา PTTOR พระรามเก้า 46 (15032),{'open_now': True},"[{'height': 3456, 'html_attributions': ['<a hr...",ChIJA1fxwJFhHTERVa981VBky1A,5.0,ChIJA1fxwJFhHTERVa981VBky1A,GOOGLE,"[convenience_store, store, point_of_interest, ...",1,"PJRG+44M, Khwaeng Suan Luang, Suan Luang",30362P,NaN,NaN
25050,OPERATIONAL,"{'location': {'lat': 13.74231, 'lng': 100.6247...",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,7-Eleven (Store 09458),{'open_now': True},"[{'height': 2250, 'html_attributions': ['<a hr...",ChIJpZBm_NlhHTER67tNN-HwhuQ,3.4,ChIJpZBm_NlhHTER67tNN-HwhuQ,GOOGLE,"[convenience_store, store, point_of_interest, ...",5,"31 Rama IX Soi 41, Suan Luang",30362P,"{'compound_code': 'PJRF+WW Bangkok', 'global_c...",NaN
25051,OPERATIONAL,"{'location': {'lat': 13.7602053, 'lng': 100.62...",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,7-Eleven สาขา รามคำแหง 57 (01442),{'open_now': True},"[{'height': 1200, 'html_attributions': ['<a hr...",ChIJEXFzMeJhHTERBaouwQpN_gQ,3.0,ChIJEXFzMeJhHTERBaouwQpN_gQ,GOOGLE,"[convenience_store, store, point_of_interest, ...",1,"QJ6C+3RM, Ramkhamhaeng Rd, Hua Mak, Bang Kapi ...",30362P,NaN,NaN
25052,OPERATIONAL,"{'locatio

#Tranform

In [ ]:
home2.columns

Index(['id', 'name_th', 'name_en', 'project_name', 'company', 'province',
       'district', 'sub_district', 'property_type', 'home improvement',
       'renovate', 'public utility', 'land type', 'land wide(m)',
       'land long(m)', 'topsoil', 'corner', 'zip_code', 'zone', 'alley',
       'street', 'latitude', 'longitude', 'number_of_rai', 'number_of_ngan',
       'number_of_va', 'construct_year', 'model', 'sell_price',
       'number_of_unit', 'number_of_story', 'number_of_bedroom',
       'number_of_bathroom', 'number_of_parking', 'usable_area',
       'central_cost', 'direction', 'furniture', 'building_number',
       'number_of_building', 'floor_number', 'parking_percentage', 'has_pool',
       'has_keycard', 'has_security', 'has_fitness', 'has_garden'],
      dtype='object')

In [ ]:
selected_columns = ['id','renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking', 'sell_price', 'district', 	'sub_district', 'zone']
home2_selected = home2[selected_columns]
home2_selected

,id,renovate,number_of_rai,number_of_ngan,number_of_va,number_of_story,number_of_bedroom,number_of_bathroom,direction,furniture,usable_area,number_of_parking,sell_price,district,sub_district,zone
0,1,no,0,1,45,2,2,2,ไม่ระบุ,ไม่มี,244,2,390000.0,ชนบท,ศรีบุญเรือง,
1,4,no,0,0,88,1,1,1,ไม่ระบุ,ไม่มี,164,3,820000.0,น้ำพอง,ทรายมูล,
2,16,no,1,0,8,1,11,11,ไม่ระบุ,ไม่ระบุ,489,9,3260000.0,มัญจาคีรี,กุดเค้า,
3,18,no,0,0,61.6,2,3,3,ไม่ระบุ,ไม่ระบุ,203,2,5000000.0,เมืองขอนแก่น,บ้านเป็ด,มะลิวัลย์-ร.8-สนามบิน
4,19,no,0,0,72.3,2,3,2,ไม่ระบุ,ไม่ระบุ,164,2,3500000.0,เมืองขอนแก่น,บ้านเป็ด,มะลิวัลย์-ร.8-สนามบิน
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,20460P,no,0,0,73,1,2,1,ไม่ระบุ,ไม่ระบุ,108.76,,437400.0,บ้านฝาง,หนองบัว,
1652,20462P,no,0,0,84,1,2,1,ไม่ระบุ,ไม่ระบุ,119,,714600.0,หนองเรือ,โนนทอง,
1653,20463P,no,0,1,45,2,1,1,ไม่ระบุ,ไม่ระบุ,188.5,,451000.0,แวงน้อย,ก้านเหลือง,
1654,20466P,no,0,1,59,2,1,1,ไม่ระบุ,ไม่ระบุ,207,,910000.0,ชุมแพ,วังหินลาด,


In [ ]:
home2_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1656 entries, 0 to 1655
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   renovate            1656 non-null   object
 1   number_of_rai       1656 non-null   object
 2   number_of_ngan      1656 non-null   object
 3   number_of_va        1656 non-null   object
 4   number_of_story     1656 non-null   object
 5   number_of_bedroom   1656 non-null   object
 6   number_of_bathroom  1656 non-null   object
 7   direction           1656 non-null   object
 8   furniture           1656 non-null   object
 9   usable_area         1656 non-null   object
 10  number_of_parking   1656 non-null   object
 11  sell_price          1656 non-null   object
 12  district            1656 non-null   object
 13  sub_district        1656 non-null   object
 14  zone                1656 non-null   object
dtypes: object(15)
memory usage: 194.2+ KB


In [ ]:
school.head()

,business_status,geometry,icon,icon_background_color,icon_mask_base_uri,name,photos,place_id,rating,reference,scope,types,user_ratings_total,vicinity,school_id,plus_code,opening_hours,permanently_closed
0,OPERATIONAL,"{'location': {'lat': 16.1311875, 'lng': 102.65...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Ban La Wa Health Center,"[{'height': 3456, 'html_attributions': ['<a hr...",ChIJW5GGVOuCGDERn2dgufu1KOU,4.5,ChIJW5GGVOuCGDERn2dgufu1KOU,GOOGLE,"[school, point_of_interest, establishment]",8,"4MJ2+F5, Mueang Phia, Ban Phai District",1,NaN,NaN,NaN
1,OPERATIONAL,"{'location': {'lat': 16.1350641, 'lng': 102.62...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Ban Wang Woen Kut Lom School,"[{'height': 3682, 'html_attributions': ['<a hr...",ChIJ1QvkXLaCGDERc-lFOXnj0NY,5.0,ChIJ1QvkXLaCGDERc-lFOXnj0NY,GOOGLE,"[primary_school, school, point_of_interest, es...",3,"8 Soi Si Bunrueang, Chonnabot, Chonnabot District",1,"{'compound_code': '4JPH+2J Si Bun Rueang, Chon...",NaN,NaN
2,OPERATIONAL,"{'location': {'lat': 16.1482073, 'lng': 102.62...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,ศูนย์พัฒนาเด็กเล็กบ้านหนองหัวช้าง,NaN,ChIJ3x63JVOdGDERRmIYy0FvMus,0.0,ChIJ3x63JVOdGDERRmIYy0FvMus,GOOGLE,"[school, point_of_interest, establishment]",0,"4JXC+78J, Nong Paen, Mancha Khiri District",1,NaN,NaN,NaN
3,OPERATIONAL,"{'location': {'lat': 16.1279195, 'lng': 102.61...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,บ้านหนองหวาย,"[{'height': 1588, 'html_attributions': ['<a hr...",ChIJG5AU2QCdGDERoKsuWds9l1w,0.0,ChIJG5AU2QCdGDERoKsuWds9l1w,GOOGLE,"[school, point_of_interest, establishment]",0,"4JH7+5G8, Si Bun Rueang, Chonnabot District",1,NaN,NaN,NaN
4,OPERATIONAL,"{'location': {'lat': 16.6744524, 'lng': 102.91...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Ban Nong Bua Ban School,"[{'height': 1327, 'html_attributions': ['<a hr...",ChIJ3z46bb7wIjERqVRwLwU9thY,5.0,ChIJ3z46bb7wIjERqVRwLwU9thY,GOOGLE,"[school, point_of_interest, establishment]",1,"MWF9+QHX, Sai Mun, Nam Phong District",4,NaN,NaN,NaN


In [ ]:
school[school['name'] == 'Ban La Wa Health Center']

,business_status,geometry,icon,icon_background_color,icon_mask_base_uri,name,photos,place_id,rating,reference,scope,types,user_ratings_total,vicinity,school_id,plus_code,opening_hours,permanently_closed,latitude,longitude
0,OPERATIONAL,"{'location': {'lat': 16.1311875, 'lng': 102.65...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Ban La Wa Health Center,"[{'height': 3456, 'html_attributions': ['<a hr...",ChIJW5GGVOuCGDERn2dgufu1KOU,4.5,ChIJW5GGVOuCGDERn2dgufu1KOU,GOOGLE,"[school, point_of_interest, establishment]",8,"4MJ2+F5, Mueang Phia, Ban Phai District",1,NaN,NaN,NaN,None,None
4060,OPERATIONAL,"{'location': {'lat': 16.1311875, 'lng': 102.65...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Ban La Wa Health Center,"[{'height': 3456, 'html_attributions': ['<a hr...",ChIJW5GGVOuCGDERn2dgufu1KOU,4.5,ChIJW5GGVOuCGDERn2dgufu1KOU,GOOGLE,"[school, point_of_interest, establishment]",8,"4MJ2+F5, Mueang Phia, Ban Phai District",1185,NaN,NaN,NaN,None,None
22128,OPERATIONAL,"{'location': {'lat': 16.1311875, 'lng': 102.65...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Ban La Wa Health Center,"[{'height': 3456, 'html_attributions': ['<a hr...",ChIJW5GGVOuCGDERn2dgufu1KOU,4.5,ChIJW5GGVOuCGDERn2dgufu1KOU,GOOGLE,"[school, point_of_interest, establishment]",8,"4MJ2+F5, Mueang Phia, Ban Phai District",18559,NaN,NaN,NaN,None,None
34091,OPERATIONAL,"{'location': {'lat': 16.1311875, 'lng': 102.65...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Ban La Wa Health Center,"[{'height': 3456, 'html_attributions': ['<a hr...",ChIJW5GGVOuCGDERn2dgufu1KOU,4.5,ChIJW5GGVOuCGDERn2dgufu1KOU,GOOGLE,"[school, point_of_interest, establishment]",8,"4MJ2+F5, Mueang Phia, Ban Phai District",20442P,NaN,NaN,NaN,None,None
36196,OPERATIONAL,"{'location': {'lat': 16.1311875, 'lng': 102.65...",https://maps.gstatic.com/mapfiles/place_api/ic...,#7B9EB0,https://maps.gstatic.com/mapfiles/place_api/ic...,Ban La Wa Health Center,"[{'height': 3456, 'html_attributions': ['<a hr...",ChIJW5GGVOuCGDERn2dgufu1KOU,4.5,ChIJW5GGVOuCGDERn2dgufu1KOU,GOOGLE,"[point_of_interest, school, establishment]",8,"4MJ2+F5, Mueang Phia, Ban Phai District",30278P,NaN,NaN,NaN,None,None


In [ ]:
school_selected = school[['geometry', 'name', 'school_id','vicinity']]
school_selected

,geometry,name,school_id,vicinity
0,"{'location': {'lat': 16.1311875, 'lng': 102.65...",Ban La Wa Health Center,1,"4MJ2+F5, Mueang Phia, Ban Phai District"
1,"{'location': {'lat': 16.1350641, 'lng': 102.62...",Ban Wang Woen Kut Lom School,1,"8 Soi Si Bunrueang, Chonnabot, Chonnabot District"
2,"{'location': {'lat': 16.1482073, 'lng': 102.62...",ศูนย์พัฒนาเด็กเล็กบ้านหนองหัวช้าง,1,"4JXC+78J, Nong Paen, Mancha Khiri District"
3,"{'location': {'lat': 16.1279195, 'lng': 102.61...",บ้านหนองหวาย,1,"4JH7+5G8, Si Bun Rueang, Chonnabot District"
4,"{'location': {'lat': 16.6744524, 'lng': 102.91...",Ban Nong Bua Ban School,4,"MWF9+QHX, Sai Mun, Nam Phong District"
...,...,...,...,...
36605,"{'location': {'lat': 13.7493223, 'lng': 100.65...",Lamsalee School Bangkapi Bangkok,30362P,"183 Krung Thep Kritha Rd, Hua Mak, Bang Kapi D..."
36606,"{'location': {'lat': 13.7332085, 'lng': 100.64...",piyajitvittaya school,30362P,"4 Phatthanakan Rd, Suan Luang"
36607,"{'location': {'lat': 13.7645946, 'lng': 100.63...",Khaisri Pramot Anusorn School,30362P,"136 Lat Phrao 134 Alley, Bangkok"
36608,"{'location': {'lat': 13.733252, 'lng': 100.640...",Khlong Kalantan School,30362P,"646 Phatthanakan Rd, Suan Luang"


In [ ]:
# prompt: นำ location ออกมาเป็น columns ของตาราง school_selected

school_selected = school_selected.join(pd.DataFrame(school_selected.pop('geometry').tolist()))

In [ ]:
school_selected

,name,school_id,vicinity,location,viewport
0,Ban La Wa Health Center,1,"4MJ2+F5, Mueang Phia, Ban Phai District","{'lat': 16.1311875, 'lng': 102.6504375}","{'northeast': {'lat': 16.13271247989272, 'lng'..."
1,Ban Wang Woen Kut Lom School,1,"8 Soi Si Bunrueang, Chonnabot, Chonnabot District","{'lat': 16.1350641, 'lng': 102.6290474}","{'northeast': {'lat': 16.13646392989272, 'lng'..."
2,ศูนย์พัฒนาเด็กเล็กบ้านหนองหัวช้าง,1,"4JXC+78J, Nong Paen, Mancha Khiri District","{'lat': 16.1482073, 'lng': 102.6207588}","{'northeast': {'lat': 16.14956522989272, 'lng'..."
3,บ้านหนองหวาย,1,"4JH7+5G8, Si Bun Rueang, Chonnabot District","{'lat': 16.1279195, 'lng': 102.6138295}","{'northeast': {'lat': 16.12916487989272, 'lng'..."
4,Ban Nong Bua Ban School,4,"MWF9+QHX, Sai Mun, Nam Phong District","{'lat': 16.6744524, 'lng': 102.9189581}","{'northeast': {'lat': 16.67573422989272, 'lng'..."
...,...,...,...,...,...
36605,Lamsalee School Bangkapi Bangkok,30362P,"183 Krung Thep Kritha Rd, Hua Mak, Bang Kapi D...","{'lat': 13.7493223, 'lng': 100.6507867}","{'northeast': {'lat': 13.75060142989272, 'lng'..."
36606,piyajitvittaya school,30362P,"4 Phatthanakan Rd, Suan Luang","{'lat': 13.7332085, 'lng': 100.6449628}","{'northeast': {'lat': 13.73461462989272, 'lng'..."
36607,Khaisri Pramot Anusorn School,30362P,"136 Lat Phrao 134 Alley, Bangkok","{'lat': 13.7645946, 'lng': 100.636494}","{'northeast': {'lat': 13.76594622989272, 'lng'..."
36608,Khlong Kalantan School,30362P,"646 Phatthanakan Rd, Suan Luang","{'lat': 13.733252, 'lng': 100.6405966}","{'northeast': {'lat': 13.73459572989272, 'lng'..."


In [ ]:
# prompt: นำ lat กับ lng ออกมาเป็น columns

# Assuming 'school_selected' DataFrame has 'location' column with lat and lng
# Splitting the 'location' column into 'lat' and 'lng' columns
school_selected[['lat', 'lng']] = pd.DataFrame(school_selected['location'].tolist(), index= school_selected.index)

# Display the updated DataFrame
school_selected.head()

,name,school_id,vicinity,location,viewport,lat,lng
0,Ban La Wa Health Center,1,"4MJ2+F5, Mueang Phia, Ban Phai District","{'lat': 16.1311875, 'lng': 102.6504375}","{'northeast': {'lat': 16.13271247989272, 'lng'...",16.131187,102.650437
1,Ban Wang Woen Kut Lom School,1,"8 Soi Si Bunrueang, Chonnabot, Chonnabot District","{'lat': 16.1350641, 'lng': 102.6290474}","{'northeast': {'lat': 16.13646392989272, 'lng'...",16.135064,102.629047
2,ศูนย์พัฒนาเด็กเล็กบ้านหนองหัวช้าง,1,"4JXC+78J, Nong Paen, Mancha Khiri District","{'lat': 16.1482073, 'lng': 102.6207588}","{'northeast': {'lat': 16.14956522989272, 'lng'...",16.148207,102.620759
3,บ้านหนองหวาย,1,"4JH7+5G8, Si Bun Rueang, Chonnabot District","{'lat': 16.1279195, 'lng': 102.6138295}","{'northeast': {'lat': 16.12916487989272, 'lng'...",16.127920,102.613829
4,Ban Nong Bua Ban School,4,"MWF9+QHX, Sai Mun, Nam Phong District","{'lat': 16.6744524, 'lng': 102.9189581}","{'northeast': {'lat': 16.67573422989272, 'lng'...",16.674452,102.918958


In [ ]:
# prompt: เปลี่ยนชื่อ columns ของ home2 latitude:lat longitude:lng

home2 = home2.rename(columns={'latitude': 'lat', 'longitude': 'lng'})

In [ ]:
# prompt: เลือก school ที่อยู่ในจังหวัดขอนแก่น หรือ khon kaen

# Filter schools in Khon Kaen province
khon_kaen_schools = school_selected[school_selected['vicinity'].str.contains('ขอนแก่น', na=False) | school_selected['vicinity'].str.contains('Khon Kaen', na=False)]

khon_kaen_schools

,name,school_id,vicinity,location,viewport,lat,lng
6,Ban Nong Wa Sukchai School,4,"Moo.12, Tambon Sai Mun, Amphoe Nam Phong, Khon...","{'lat': 16.682199, 'lng': 102.909279}","{'northeast': {'lat': 16.68360702989272, 'lng'...",16.682199,102.909279
15,โรงเรียนบ้านไผ่ประถมศึกษา,6,"หมู่ 1 คุ้มจัดสรร, ตำบลในเมือง อำเภอบ้านไผ่ จั...","{'lat': 16.0706613, 'lng': 102.7329711}","{'northeast': {'lat': 16.07201652989272, 'lng'...",16.070661,102.732971
24,โรงเรียนบ้านหนองน้ำใส,6,"หมู่ที่ 3, ตำบลหนองน้ำใส อำเภอบ้านไผ่ จังหวัดข...","{'lat': 16.0436701, 'lng': 102.7384617}","{'northeast': {'lat': 16.04502117989272, 'lng'...",16.043670,102.738462
26,โรงเรียนบ้านหนองหิน,6,"บ้านศิลา-นาโพธิ์, ต.ในเมือง อ.บ้านไผ่. จ.ขอนแก...","{'lat': 16.0687492, 'lng': 102.7357992}","{'northeast': {'lat': 16.07002082989272, 'lng'...",16.068749,102.735799
29,Mahathai Sueksa Kindergarten,6,"เลขที่ 641/61, ถ.เจนจบทิศ, ต.ในเมือง อ.บ้านไผ่...","{'lat': 16.0697929, 'lng': 102.7340638}","{'northeast': {'lat': 16.07116482989272, 'lng'...",16.069793,102.734064
...,...,...,...,...,...,...,...
36585,กองกิจการนักศึกษา มหาวิทยาลัยขอนแก่น,30360P,"16.47802,102.82052, เมืองขอนแก่น","{'lat': 16.47801, 'lng': 102.8203845}","{'northeast': {'lat': 16.47927452989272, 'lng'...",16.478010,102.820385
36586,สถาบันภาษาไอเฮ้าส์ iHouse School,30360P,"140, 350 Kanlapaphruek Rd, Tambon Nai Mueang, ...","{'lat': 16.4647151, 'lng': 102.8247331}","{'northeast': {'lat': 16.46610522989273, 'lng'...",16.464715,102.824733
36587,สถาบันตรงใจติวเตอร์,30360P,233/11 หมู่ 12 Mueang Khon Kaen District,"{'lat': 16.4806252, 'lng': 102.8199806}","{'northeast': {'lat': 16.48194247989272, 'lng'...",16.480625,102.819981
36589,ห้องสมุดคณะแพทยศาสตร์ มข.,30360P,"FR9J+M42, Nai Mueang, Mueang Khon Kaen District","{'lat': 16.4691493, 'lng': 102.8302428}","{'northeast': {'lat': 16.47035532989272, 'lng'...",16.469149,102.830243


In [ ]:
home_es

,ID ผู้ประเมิน,post id,ID,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์
0,1,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,"2,890,000","2,500,000","5,000-15,000",,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1,2,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,"2,890,000","2,700,000","โครงการนี้ราคาเสนอขาย 2,500,000-3,000,000 บาท/...",,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
2,1,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,"2,600,000","2,200,000","5,000-15,000",,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
3,2,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,"2,600,000","2,400,000","ราคาเสนอขายประมาณ 2,200,000 - 2,500,000 บาท/หลัง",,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
4,1,https://nayoo.co/khonkaen/posts/495,495,บ้านเดี่ยว,มั่นใจ,"6,050,000","5,000,000","5,000-15,000",,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
...,...,...,...,...,...,...,...,...,...,...
1912,2,https://nayoo.co/khonkaen/posts/17867,17867,บ้านเดี่ยว,มั่นใจ,"965,000","550,000","ราคาเสนอขาย 2,000 - 3,000 บาท/ตารางวา",,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1913,2,https://nayoo.co/khonkaen/posts/17873,17873,บ้านเดี่ยว,มั่นใจ,"1,000,000","1,000,000","ราคาเสนอขาย 2,500 - 3,000 บาท/ตารางวา",,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1914,2,https://nayoo.co/khonkaen/posts/17874,17874,บ้านเดี่ยว,มั่นใจ,"10,873,000","10,000,000","ราคาเสนอขาย 12,000 - 15,000 บาท/ตารางวา",,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1915,2,https://nayoo.co/khonkaen/posts/17876,17876,บ้านเดี่ยว,มั่นใจ,"1,000,000","1,200,000",ราคาเสนอขาย 1.2-1.5 ล้านบาท/หลัง,,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน


In [ ]:
home_es_01 = home_es[home_es['ID'] == '01']

In [ ]:
home_es_01

,ID ผู้ประเมิน,post id,ID,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,estimator,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,score
0,1,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,"2,890,000","2,500,000","5,000-15,000",,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
2,1,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,"2,600,000","2,200,000","5,000-15,000",,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
4,1,https://nayoo.co/khonkaen/posts/495,495,บ้านเดี่ยว,มั่นใจ,"6,050,000","5,000,000","5,000-15,000",,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
6,1,https://nayoo.co/khonkaen/posts/564,564,บ้านเดี่ยว,มั่นใจ,"4,400,000","3,500,000","5,000-18,000",,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
8,1,https://nayoo.co/khonkaen/posts/566,566,บ้านเดี่ยว,มั่นใจ,"2,750,000","2,500,000","5,000-25,000",,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
...,...,...,...,...,...,...,...,...,...,...
1862,1,https://www.baania.com/listing/%E0%B8%82%E0%B8...,30327P,บ้านเดี่ยว,มั่นใจ,"4,950,000","4,500,000",เพื่อป้องกันปัญหาที่อาจเกิดขึ้นภายหลังหากต้องม...,,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
1863,1,https://www.baania.com/listing/%E0%B8%82%E0%B8...,30328P,บ้านเดี่ยว,มั่นใจ,"3,950,000","3,500,000",เพื่อป้องกันปัญหาที่อาจเกิดขึ้นภายหลังหากต้องม...,,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
1864,1,https://www.baania.com/listing/%E0%B8%82%E0%B8...,30330P,บ้านเดี่ยว,มั่นใจ,"1,700,000","1,800,000",เพื่อป้องกันปัญหาที่อาจเกิดขึ้นภายหลังหากต้องม...,,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
1865,1,https://www.baania.com/listing/%E0%B8%82%E0%B8...,30331P,บ้านเดี่ยว,มั่นใจ,"5,900,000","5,500,000",เพื่อป้องกันปัญหาที่อาจเกิดขึ้นภายหลังหากต้องม...,,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี


In [ ]:
home2_selected.columns

Index(['id', 'renovate', 'number_of_rai', 'number_of_ngan', 'number_of_va',
       'number_of_story', 'number_of_bedroom', 'number_of_bathroom',
       'direction', 'furniture', 'usable_area', 'number_of_parking',
       'sell_price', 'district', 'sub_district', 'zone'],
      dtype='object')

In [ ]:
home_es_01 = home_es_01.rename(columns={'ID': 'id'}) # Assuming 'ID' is the original column name in home_es_01

# Merge the two dataframes
merged_df = pd.merge(home2_selected, home_es_01, on='id', how='left')

In [ ]:
merged_df

,id,renovate,number_of_rai,number_of_ngan,number_of_va,number_of_story,number_of_bedroom,number_of_bathroom,direction,furniture,...,zone,ID ผู้ประเมิน,post id,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,estimator,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,score
0,1,no,0,1,45,2,2,2,ไม่ระบุ,ไม่มี,...,,1,https://nayoo.co/khonkaen/posts/1,บ้านเดี่ยว,มั่นใจ,"390,000","600,000","3,000-30,000 บาท/ตารางวา",,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
1,4,no,0,0,88,1,1,1,ไม่ระบุ,ไม่มี,...,,1,https://nayoo.co/khonkaen/posts/4,บ้านเดี่ยว,มั่นใจ,"930,000","750,000","2,000-10,000 บาท/ตารางวา",,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
2,16,no,1,0,8,1,11,11,ไม่ระบุ,ไม่ระบุ,...,,1,https://nayoo.co/khonkaen/posts/16,บ้านเดี่ยว,มั่นใจ,"2,994,000","2,800,000","2,000-10,000 บาท/ตารางวา",,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
3,18,no,0,0,61.6,2,3,3,ไม่ระบุ,ไม่ระบุ,...,มะลิวัลย์-ร.8-สนามบิน,1,https://nayoo.co/khonkaen/posts/18,บ้านเดี่ยว,มั่นใจ,"5,300,000","4,500,000","3,000-60,000 บาท/ตารางวา",,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
4,19,no,0,0,72.3,2,3,2,ไม่ระบุ,ไม่ระบุ,...,มะลิวัลย์-ร.8-สนามบิน,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1667,20460P,no,0,0,73,1,2,1,ไม่ระบุ,ไม่ระบุ,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1668,20462P,no,0,0,84,1,2,1,ไม่ระบุ,ไม่ระบุ,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1669,20463P,no,0,1,45,2,1,1,ไม่ระบุ,ไม่ระบุ,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1670,20466P,no,0,1,59,2,1,1,ไม่ระบุ,ไม่ระบุ,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# prompt: เติมค่า sub_district ที่ว่าง ด้วย district

# Fill missing 'sub_district' values with 'district' values
merged_df['sub_district'].fillna(merged_df['district'], inplace=True)

<ipython-input-140-7539fa6b165d>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['sub_district'].fillna(merged_df['district'], inplace=True)


In [ ]:
merged_df['sub_district'].isna().sum()

0

In [ ]:
merged_df['sub_district'].unique()

array(['ศรีบุญเรือง', 'ทรายมูล', 'กุดเค้า', 'บ้านเป็ด', 'ศิลา', 'ในเมือง',
       'เมืองเก่า', 'คอนฉิม', 'บ้านดง', 'เมืองพล', 'หนองโก', 'เขาสวนกวาง',
       'ห้วยแก', 'ชุมแพ', 'บ้านขาม', 'ม่วงหวาน', 'สะอาด', 'หนองน้ำใส',
       'เมืองเพีย', 'พระยืน', 'เพ็กใหญ่', 'เก่างิ้ว', 'บ้านทุ่ม', 'โคกสี',
       'พระลับ', 'ท่าพระ', 'หนองตูม', 'ท่านางแนว', 'ท่าวัด',
       'ใหม่นาเพียง', 'ดอนดู่', 'คึมชาด', 'หนองไผ่', 'น้ำพอง', 'บ้านฝาง',
       'ชนบท', 'วังหินลาด', 'กุดธาตุ', 'ยางคำ', 'บ้านเม็ง', 'คูคำ',
       'กุดเพียขอม', 'ป่ามะนาว', 'หนองแดง', 'หนองแวง', 'บ้านโต้น',
       'หนองกุงธนสาร', 'บ้านแฮด', 'จระเข้', 'ดอนหัน', 'ห้วยยาง',
       'หนองแซง', 'แดงใหญ่', 'หนองแวงนางเบ้า', 'สีชมพู', 'หนองบัว',
       'บ้านเหล่า', 'โคกงาม', 'ขามป้อม', 'หนองเรือ', 'โนนสะอาด',
       'ขัวเรียง', 'นาหนองทุ่ม', 'ไชยสอ', 'ท่ากระเสริม', 'พังทุย',
       'วังชัย', 'กุดน้ำใส', 'ลอมคอม', 'โจดหนองแก', 'สำโรง', 'โนนธาตุ',
       'หนองกุงเซิน', 'กุดขอนแก่น', 'สงเปือย', 'ภูเวียง', 'สวนหม่อน',
       'หนองแปน', 'ปอแดง', 

In [ ]:
merged_df['score'] = merged_df['score'].astype(str).str.extract('(\d+)').astype(float).fillna(0).astype(int) # Changed to float first, then fillna with 0, then convert to int


In [ ]:
merged_df

,id,renovate,number_of_rai,number_of_ngan,number_of_va,number_of_story,number_of_bedroom,number_of_bathroom,direction,furniture,...,zone,ID ผู้ประเมิน,post id,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,estimator,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,score
0,1,no,0,1,45,2,2,2,ไม่ระบุ,ไม่มี,...,,1,https://nayoo.co/khonkaen/posts/1,บ้านเดี่ยว,มั่นใจ,"390,000","600,000","3,000-30,000 บาท/ตารางวา",,4
1,4,no,0,0,88,1,1,1,ไม่ระบุ,ไม่มี,...,,1,https://nayoo.co/khonkaen/posts/4,บ้านเดี่ยว,มั่นใจ,"930,000","750,000","2,000-10,000 บาท/ตารางวา",,3
2,16,no,1,0,8,1,11,11,ไม่ระบุ,ไม่ระบุ,...,,1,https://nayoo.co/khonkaen/posts/16,บ้านเดี่ยว,มั่นใจ,"2,994,000","2,800,000","2,000-10,000 บาท/ตารางวา",,2
3,18,no,0,0,61.6,2,3,3,ไม่ระบุ,ไม่ระบุ,...,มะลิวัลย์-ร.8-สนามบิน,1,https://nayoo.co/khonkaen/posts/18,บ้านเดี่ยว,มั่นใจ,"5,300,000","4,500,000","3,000-60,000 บาท/ตารางวา",,2
4,19,no,0,0,72.3,2,3,2,ไม่ระบุ,ไม่ระบุ,...,มะลิวัลย์-ร.8-สนามบิน,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1667,20460P,no,0,0,73,1,2,1,ไม่ระบุ,ไม่ระบุ,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1668,20462P,no,0,0,84,1,2,1,ไม่ระบุ,ไม่ระบุ,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1669,20463P,no,0,1,45,2,1,1,ไม่ระบุ,ไม่ระบุ,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1670,20466P,no,0,1,59,2,1,1,ไม่ระบุ,ไม่ระบุ,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
home2.columns

Index(['id', 'name_th', 'name_en', 'project_name', 'company', 'province',
       'district', 'sub_district', 'property_type', 'home improvement',
       'renovate', 'public utility', 'land type', 'land wide(m)',
       'land long(m)', 'topsoil', 'corner', 'zip_code', 'zone', 'alley',
       'street', 'lat', 'lng', 'number_of_rai', 'number_of_ngan',
       'number_of_va', 'construct_year', 'model', 'sell_price',
       'number_of_unit', 'number_of_story', 'number_of_bedroom',
       'number_of_bathroom', 'number_of_parking', 'usable_area',
       'central_cost', 'direction', 'furniture', 'building_number',
       'number_of_building', 'floor_number', 'parking_percentage', 'has_pool',
       'has_keycard', 'has_security', 'has_fitness', 'has_garden', 'estimator',
       'score'],
      dtype='object')

In [ ]:
home_loc = home2[['id', 'lat', 'lng']]
home_loc

,id,lat,lng
0,1,16.139321,102.631587
1,4,16.6761557,102.9179977
2,16,16.129358,102.53067
3,18,16.4590524,102.8064042
4,19,16.45,102.80068
...,...,...,...
1651,20460P,16.48011,102.574033
1652,20462P,16.528551,102.411762
1653,20463P,15.867444,102.482085
1654,20466P,16.636912,102.105343


In [ ]:
seven_loc = seven[['seven_id', 'lat', 'lng']]
seven_loc

,seven_id,lat,lng
0,6,16.060315,102.726486
1,6,16.056474,102.726844
2,6,16.059541,102.729431
3,6,16.075200,102.735602
4,6,16.070648,102.734405
...,...,...,...
25049,30362P,13.740390,100.625289
25050,30362P,13.742310,100.624788
25051,30362P,13.760205,100.622040
25052,30362P,13.736292,100.616910


In [ ]:
school_loc = school_selected[['school_id', 'lat', 'lng']]
school_loc

,school_id,lat,lng
0,1,16.131187,102.650437
1,1,16.135064,102.629047
2,1,16.148207,102.620759
3,1,16.127920,102.613829
4,4,16.674452,102.918958
...,...,...,...
36605,30362P,13.749322,100.650787
36606,30362P,13.733208,100.644963
36607,30362P,13.764595,100.636494
36608,30362P,13.733252,100.640597


In [ ]:
# prompt: จำนวน 7-11 และ โรงเรียน ในรัศมี 1 กิโลเมตร (ตั้งชื่อ 'd1000-7/11' และ 'd1000-school')

import pandas as pd
import numpy as np

# Assuming home_loc, seven_loc, and school_loc DataFrames are already defined as in your previous code.
# ... (Your existing code for loading and preprocessing data)

def distance(lat1, lon1, lat2, lon2):
    # Calculate the distance between two coordinates using Haversine formula
    # Convert latitude and longitude to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [float(lat1), float(lon1), float(lat2), float(lon2)]) # Convert to float before applying np.radians

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Radius of earth in kilometers. Use 3956 for miles
    return c * r

# Function to count nearby locations within a radius
def count_nearby_locations(home_id, home_lat, home_lng, locations_df, radius):
    count = 0
    for _, row in locations_df.iterrows():
        dist = distance(home_lat, home_lng, row['lat'], row['lng'])
        if dist <= radius:
            count += 1
    return count

# Ensure 'lat' and 'lng' columns are numeric in all DataFrames
home_loc['lat'] = pd.to_numeric(home_loc['lat'], errors='coerce')
home_loc['lng'] = pd.to_numeric(home_loc['lng'], errors='coerce')
seven_loc['lat'] = pd.to_numeric(seven_loc['lat'], errors='coerce')
seven_loc['lng'] = pd.to_numeric(seven_loc['lng'], errors='coerce')
school_loc['lat'] = pd.to_numeric(school_loc['lat'], errors='coerce')
school_loc['lng'] = pd.to_numeric(school_loc['lng'], errors='coerce')

# Apply the function to count nearby 7-11s and schools
home_loc['d1000-7/11'] = home_loc.apply(lambda row: count_nearby_locations(row['id'], row['lat'], row['lng'], seven_loc, 1), axis=1)
home_loc['d1000-school'] = home_loc.apply(lambda row: count_nearby_locations(row['id'], row['lat'], row['lng'], school_loc, 1), axis=1)

# Now 'home_loc' DataFrame contains 'd1000-7/11' and 'd1000-school' columns

# Merge with the main dataframe
merged_df = pd.merge(merged_df, home_loc[['id', 'd1000-7/11', 'd1000-school']], on='id', how='left')
merged_df

<ipython-input-165-3e5dbd054823>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home_loc['lat'] = pd.to_numeric(home_loc['lat'], errors='coerce')
<ipython-input-165-3e5dbd054823>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home_loc['lng'] = pd.to_numeric(home_loc['lng'], errors='coerce')
<ipython-input-165-3e5dbd054823>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

KeyboardInterrupt: 

# 2 จากตารางในข้อ 1 ตอบคำถามต่อไปนี้ (5 คะแนน)
1. zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)
2. บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่ (2 คะแนน)
3. ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)

zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)

In [ ]:
# Convert 'sell_price' column to numeric, handling errors
home2['sell_price'] = pd.to_numeric(home2['sell_price'], errors='coerce')

# Group by 'zone' and calculate the mean 'sell_price'
avg_sell_price_by_zone = home2.groupby('zone')['sell_price'].mean()

# Find the zone with the highest average 'sell_price'
zone_with_highest_avg_price = avg_sell_price_by_zone.idxmax()

print(f"Zone with the highest average selling price: {zone_with_highest_avg_price}")

Zone with the highest average selling price: ในเมือง-รื่นรมย์-เทพารักษ์


ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด

In [ ]:
avg_score_by_subdistrict = merged_df.groupby('sub_district')['score'].mean()

# Find the sub_district with the highest average 'score'
subdistrict_with_highest_avg_score = avg_score_by_subdistrict.idxmax()

print(f"Subdistrict with the highest average score: {subdistrict_with_highest_avg_score}")

Subdistrict with the highest average score: ศรีบุญเรือง
